In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the desired web page
driver.get('https://www.resonance.ac.in/results/jee-Advanced-2020.aspx')

def scrape_table():
    try:
        # Wait for the table content to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, 'grvResult_yccp'))
        )
    except TimeoutException:
        print("TimeoutException: Table not found.")
        driver.quit()
        raise
    
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # Extract table rows
    rows = soup.select('#grvResult_yccp tr')
    data = []
    for row in rows:
        row_data = []
        columns = row.find_all('td')
        for column in columns:
            # Check if the column contains an <img> tag
            img_tag = column.find('img')
            if img_tag:
                row_data.append(img_tag['src'])
            else:
                row_data.append(column.text.strip())
        data.append(row_data)
    return data

all_data = []

# Start with the first page
current_page = 1

while True:
    # Scrape the current page's table
    page_data = scrape_table()
    all_data.extend(page_data)

    try:
        if current_page % 5 == 0:
            # Find all "..." buttons
            dots_buttons = driver.find_elements(By.LINK_TEXT, '...')
            if len(dots_buttons) > 1:
                # Click the second "..." button if there are two
                next_button = dots_buttons[1]
            else:
                # Otherwise, click the first "..." button
                next_button = dots_buttons[0]
        else:
            # Find the next page button
            next_button = driver.find_element(By.LINK_TEXT, str(current_page + 1))
        
        next_button.click()
        current_page += 1
        
        # Wait for the new page to load
        WebDriverWait(driver, 20).until(
            EC.staleness_of(next_button)
        )
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, 'grvResult_yccp'))
        )
    except Exception as e:
        print(f"Exception: {e}")
        break

# Close the browser
driver.quit()

# Print or process the scraped data
for item in all_data:
    print(item)


Exception: Message: element click intercepted: Element <a href="javascript:__doPostBack('grvResult_yccp','Page$30')">...</a> is not clickable at point (127, 49). Other element would receive the click: <div class="wsmobileheader clearfix ">...</div>
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF6AA45EEA2+31554]
	(No symbol) [0x00007FF6AA3D7ED9]
	(No symbol) [0x00007FF6AA29872A]
	(No symbol) [0x00007FF6AA2F012E]
	(No symbol) [0x00007FF6AA2EDAF2]
	(No symbol) [0x00007FF6AA2EAF8B]
	(No symbol) [0x00007FF6AA2EA156]
	(No symbol) [0x00007FF6AA2DC151]
	(No symbol) [0x00007FF6AA30D02A]
	(No symbol) [0x00007FF6AA2DBA76]
	(No symbol) [0x00007FF6AA30D240]
	(No symbol) [0x00007FF6AA32C977]
	(No symbol) [0x00007FF6AA30CDD3]
	(No symbol) [0x00007FF6AA2DA33B]
	(No symbol) [0x00007FF6AA2DAED1]
	GetHandleVerifier [0x00007FF6AA768B1D+3217341]
	GetHandleVerifier [0x00007FF6AA7B5AE3+3532675]
	GetHandleVerifier [0x00007FF6AA7AB0E0+3489152]
	GetHandleVerifier [0x00007FF6AA5

In [27]:
import csv

# Assuming you have a list called 'data' that you want to write to a CSV file

# Specify the file path and name
file_path = 'finals_with_url.csv'

# Open the file in write mode
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    writer.writerows(all_data)

print("Data written to", file_path)

Data written to finals_with_url.csv
